In [1]:
import numpy as np
from numpy import nan
import pandas as pd

In [21]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'conversions')
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')

def create_dict_key(dataframe):
    '''
    '''
    conversion_key = dict()
    for year in dataframe.columns[1:]:
        values = list(dataframe[year])
        categories = list(dataframe['categories'])
        temp = dict()
        for tup in tuple(zip(categories, values)):
            temp[tup[0]] = tup[1]
        
        temp['USD'] = 1
        conversion_key[year] = temp
            
    return conversion_key

conversion_key = create_dict_key(key)


def adjustments(raw_data, conversion_key, fy_starts=13):
    '''
    Takes inputs of raw_data and conversion key and
    converts to real money terms.
    
    Inputs:
        raw_data: pandas dataframe of raw data Excel
        raw_data: pandas dataframe of raw data Excel
        
    Outputs:
        adjusted_data: pandas dataframe of adjusted data
    '''
    columns = list(raw_data.columns[fy_starts:])
    raw_data['Region'] = raw_data['Type']
        
    for index, row in raw_data.iterrows():
        currency = row['Currency']
        for year in columns:
            year_match = int(year[2:])
            infl = conversion_key[year_match]['Inflation Adjustment']
            curr = conversion_key[year_match][currency]
            if currency == 'Yen':
                row[year] = (float(row[year])/(curr)*1000)*infl
            elif currency == 'Forint':
                row[year] = (float(row[year])/(curr)/1000000)*infl
            else:
                row[year] = (float(row[year])/curr)*infl
            
        raw_data.loc[index] = row
        
    clean = raw_data
    clean['Source'] = clean['Source'].str.replace(" ", "")
    
    return clean[['Proper Name', 'Source'] + columns]

clean = adjustments(df, conversion_key)

In [4]:
clean

,Proper Name,Source,FY1991,FY1992,FY1993,FY1994,FY1995,FY1996,FY1997,FY1998,...,FY2029,FY2030,FY2031,FY2032,FY2033,FY2034,FY2035,FY2036,FY2037,FY2038
0,Abraxane,Abraxane1,0,0,0,0,0,0,0,0,...,78.256934,72.267018,57.066113,48.714287,57.176730,NaN,NaN,NaN,NaN,NaN
1,Abraxane,Abraxane2,0,0,0,0,0,0,0,0,...,124.916543,114.253278,88.536561,100.120454,98.940788,103.983509,102.943673,NaN,NaN,NaN
2,Alimta,Alimta1,0,0,0,0,0,0,0,0,...,101.383664,97.932758,91.373897,82.078003,78.093416,94.159192,76.722430,NaN,NaN,NaN
3,Alimta,Alimta2,0,0,0,0,0,0,0,0,...,41.399011,40.807584,36.807734,31.478740,33.615292,44.659269,40.034886,NaN,NaN,NaN
4,Amitiza,Amitiza1,0,0,0,0,0,0,0,0,...,27.521377,26.243063,25.010734,24.615897,13.029604,12.437349,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Tyvaso,Tyvaso2,0,0,0,0,0,0,0,0,...,2210.271250,2160.475125,789.123223,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,Xifaxan,Xifaxan1,0,0,0,0,0,0,0,0,...,125.004927,58.242191,63.381610,58.712571,80.072836,80.072836,80.072836,NaN,NaN,NaN
81,Xolair,Xolair1,0,0,0,0,0,0,0,0,...,797.514686,754.847197,742.875792,958.444861,1082.885530,1146.483442,1545.828928,NaN,NaN,NaN
82,Xolair,Xolair2,0,0,0,0,0,0,0,0,...,638.448484,568.175109,519.835415,271.948700,245.901127,223.874431,77.439704,NaN,NaN,NaN


In [22]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'dictionary')

def split_strings(string):
    '''
    '''
    new_string = '{'
    if string == '':
        return string
    for char in string:
        if char not in ['+', '-', '*', '/', '(', ')']:
            new_string += char
        else:
            new_string += '}'
            new_string += char
            new_string += '{'
    
    new_string += '}'
    new_string = new_string.replace("{}", "" )
    
    return new_string.replace("{0.5}", "0.5" )


def formulas(dataframe):
    '''
    '''
    dataframe['formula'] = dataframe['formula'].str.replace(" ", "")
    dataframe["formula"] = dataframe["formula"].apply(split_strings)
    formula_key = dict()
    for _, row in dataframe.iterrows():
        formula = row['formula']
        formula_key[row['ID']] = formula

    return formula_key

formula_key = formulas(key)

In [9]:
formula_key

{'Abraxane WW': '{Abraxane1}',
 'Abraxane US': '{Abraxane2}',
 'Abraxane Ex-US': '{Abraxane1}-{Abraxane2}',
 'Alimta WW': '{Alimta1}',
 'Alimta US': '{Alimta2}',
 'Alimta Ex-US': '{Alimta1}-{Alimta2}',
 'Amitiza WW': '{Amitiza1}',
 'Amitiza US': '{Amitiza2}',
 'Amitiza Ex-US': '{Amitiza1}-{Amitiza2}',
 'Aranesp WW': '{Aranesp1}',
 'Aranesp US': '{Aranesp2}',
 'Aranesp Ex-US': '{Aranesp1}-{Aranesp2}',
 'Auryxia WW': '{Auryxia1}',
 'Auryxia US': '{Auryxia2}',
 'Auryxia Ex-US': '{Auryxia1}-{Auryxia2}',
 'Avonex WW': '{Avonex1}',
 'Avonex US': '{Avonex2}',
 'Avonex Ex-US': '{Avonex1}-{Avonex2}',
 'Botox WW': '{Botox1}',
 'Botox US': '{Botox2}',
 'Botox Ex-US': '{Botox1}-{Botox2}',
 'Brovana WW': '{Brovana1}',
 'Brovana US': '{Brovana2}',
 'Brovana Ex-US': '{Brovana1}-{Brovana2}',
 'Byetta WW': '{Byetta1}',
 'Byetta US': '{Byetta2}',
 'Byetta Ex-US': '{Byetta1}-{Byetta2}',
 'Bydureon WW': '{Bydureon1}',
 'Bydureon US': '{Bydureon2}',
 'Bydureon Ex-US': '{Bydureon1}-{Bydureon2}',
 'Byetta_By

In [23]:
def evaluate_helper(clean_dataframe):
    '''
    '''
    holder = dict()
    for index, row in clean_dataframe.iterrows():
        for year in clean_dataframe.columns[2:]:
            if year not in holder:
                holder[year] = {row['Source'] : row[year]}
            else:
                temp = holder.get(year)
                holder[year].update({row['Source'] : row[year]})
                
    return holder

    
eval_map = evaluate_helper(clean)

def evaluate(eval_map, formula_key):
    '''
    '''
    return_dict = dict()
    for year in eval_map.keys():
        temp = eval_map.get(year)
        temp_dict = dict()
        for ID, formula in formula_key.items():
            if formula != '{Nan}':
                formula = formula.format(**temp)
                if formula != np.nan:
                    try:
                        result = eval(formula)
                    except ZeroDivisionError:
                        result = 0
                else:
                    result = formula
                temp_dict[ID] = result
            else:
                temp_dict[ID] = 'Nan'
                
        return_dict[year] = temp_dict
        
    return return_dict

evaluated_key = evaluate(eval_map, formula_key)


def turn_into_dataframe(evaluated_key):
    '''
    '''
    counter = 0 
    for year, values in evaluated_key.items():
        data = []
        for key, amount in values.items():
            data.append([key, amount])
        df_temp = pd.DataFrame(data, columns = ['Product Name', year])
        if counter > 0:
            df = df.merge(df_temp, how='inner', on='Product Name')
        else: 
            df = df_temp

        counter += 1
        
    return df
    
    
evaluated_key = turn_into_dataframe(evaluated_key)

In [41]:
evaluated_key[evaluated_key['Proper Name'] == 'Byetta']

,Product Name,FY1991,FY1992,FY1993,FY1994,FY1995,FY1996,FY1997,FY1998,FY1999,...,FY2030,FY2031,FY2032,FY2033,FY2034,FY2035,FY2036,FY2037,FY2038,Proper Name
24,Byetta WW,0,0,0,0,0,0,0,0,0,...,27.906548,26.700465,24.745944,25.066447,25.5044,26.041349,26.665047,NaN,NaN,Byetta
25,Byetta US,0,0,0,0,0,0,0,0,0,...,7.483619,6.748436,0.718824,0.500781,0.348877,0.243051,0.169326,NaN,NaN,Byetta
26,Byetta Ex-US,0,0,0,0,0,0,0,0,0,...,20.422929,19.952029,24.027119,24.565666,25.155523,25.798298,26.495721,NaN,NaN,Byetta
30,Byetta_Bydureon WW,0,0,0,0,0,0,0,0,0,...,27.906548,26.700465,24.745944,25.066447,25.5044,26.041349,26.665047,NaN,NaN,Byetta
31,Byetta_Bydureon US,0,0,0,0,0,0,0,0,0,...,7.483619,6.748436,0.718824,0.500781,0.348877,0.243051,0.169326,NaN,NaN,Byetta
32,Byetta_Bydureon Ex-US,0,0,0,0,0,0,0,0,0,...,20.422929,19.952029,24.027119,24.565666,25.155523,25.798298,26.495721,NaN,NaN,Byetta


In [56]:
def check_string(x, string_list):
    for string in string_list:
        if string in x:
            return string
    return None



def merge_final_clean(raw_dataframe, evaluated_key):
    '''
    '''
    cols_y = list(raw_dataframe.columns[13:])
    
    cols = ['Proper Name', 'Generic Name', 
            'Medicare Spend', 'Original Manufacturer', 
            'Application', 'Approval Date', 'Year']
    
    pnames = list(raw_dataframe['Proper Name'])
    temp = raw_dataframe[cols].drop_duplicates()
    
    evaluated_key["Proper Name"] = evaluated_key["Product Name"].apply(lambda x: check_string(x, pnames))
    
    final = temp.merge(evaluated_key, how='left', on='Proper Name')

    for col in cols_y:
        final = final.rename(columns={col: col[2:]})
        
    final['Proper Name'] = final['Product Name'].str.split().str.get(0)
    
    return final

    
final = merge_final_clean(df, evaluated_key)


In [57]:
final

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,1991,1992,...,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038
0,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,Abraxane WW,0,0,...,78.256934,72.267018,57.066113,48.714287,57.17673,NaN,NaN,NaN,NaN,NaN
1,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,Abraxane US,0,0,...,124.916543,114.253278,88.536561,100.120454,98.940788,103.983509,102.943673,NaN,NaN,NaN
2,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,Abraxane Ex-US,0,0,...,-46.659609,-41.986259,-31.470449,-51.406167,-41.764058,NaN,NaN,NaN,NaN,NaN
3,Alimta,Pemetrexed Disodium,4.985018e+08,Lilly,NDA,2004-02-04,2004,Alimta WW,0,0,...,101.383664,97.932758,91.373897,82.078003,78.093416,94.159192,76.72243,NaN,NaN,NaN
4,Alimta,Pemetrexed Disodium,4.985018e+08,Lilly,NDA,2004-02-04,2004,Alimta US,0,0,...,41.399011,40.807584,36.807734,31.47874,33.615292,44.659269,40.034886,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Xolair,Omalizumab,3.997580e+08,Roche,BLA,2003-06-20,2003,Xolair US,0,0,...,638.448484,568.175109,519.835415,271.9487,245.901127,223.874431,77.439704,NaN,NaN,NaN
131,Xolair,Omalizumab,3.997580e+08,Roche,BLA,2003-06-20,2003,Xolair Ex-US,0,0,...,159.066202,186.672088,223.040376,686.496161,836.984402,922.609011,1468.389224,NaN,NaN,NaN
132,Xyrem,Sodium Oxybate,2.871436e+08,Jazz,NDA,2002-07-17,2002,Xyrem WW,0,0,...,81.329613,80.697779,97.17621,100.336963,68.537847,66.740945,45.887455,NaN,NaN,NaN
133,Xyrem,Sodium Oxybate,2.871436e+08,Jazz,NDA,2002-07-17,2002,Xyrem US,0,0,...,81.329613,80.697779,97.17621,100.336963,68.537847,66.740945,45.887455,NaN,NaN,NaN


In [44]:
def reformat_final(dataframe):
    '''
    '''
    data = []
    years = dataframe.columns[8:]
    other_info = dataframe.columns[:8]
    
    for index, row in dataframe.iterrows():
        new_data = list(row[other_info].values)
        for year in years:
            if int(year) >= row['Year']:
                new_data.append(row[str(year)])
                
        data.append(new_data)
        
    longest_list = max(data, key=lambda x: len(x))
    num_years = len(longest_list) - 8
    year_cols = [f't{i}' for i in range(1, num_years+1)]
    cols = list(other_info) + year_cols
        
    data = pd.DataFrame(data, columns = cols)
    data[year_cols] = data[year_cols].astype(float)
    #data['t31'] = np.nan

    return data.fillna(np.nan)


re_final = reformat_final(final)

In [45]:
def find_last_valid(row):
    last_valid = row.last_valid_index()
    return (last_valid, row[last_valid])

def pro_rate(dataframe):
    '''
    '''
    last_values = dataframe.apply(find_last_valid, axis=1).tolist()
    
    end_of_year = pd.to_datetime(dataframe['Approval Date'].dt.year.astype(str) + '-12-31')
    days_remaining = (end_of_year - dataframe['Approval Date']).dt.days
    dataframe['percent_remaining'] = (days_remaining / 365)
    dataframe['percent_missing'] = 1 - dataframe['percent_remaining']
    
    for year in range(1,31):
        var = 't' + str(year)
        var1 = 't' + str(year+1)
        dataframe[var] = (dataframe[var]) + (dataframe[var1]*dataframe['percent_missing'])
        dataframe[var1] -= (dataframe[var1]*dataframe['percent_missing'])
    
    for i, (col, val) in enumerate(last_values):
        dataframe.loc[dataframe.index[i], col] = val
    
    return dataframe


pro_rate = pro_rate(re_final)
pro_rate

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t41,t42,t43,t44,t45,t46,t47,t48,percent_remaining,percent_missing
0,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980822,0.019178
1,Alimta,Pemetrexed Disodium,4.985018e+08,Lilly,NDA,2004-02-04,2004,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.906849,0.093151
2,Amitiza,Lubiprostone,2.793228e+08,Takeda,NDA,2006-01-31,2006,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.915068,0.084932
3,Aranesp,Darbepoetin Alfa In Polysorbat,2.116430e+08,Amgen,BLA,2001-09-17,2001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.287671,0.712329
4,Auryxia,NaN,NaN,Akebia,NDA,2014-09-05,2014,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.320548,0.679452
5,Avonex,Interferon Beta-1a,NaN,Biogen,BLA,1996-05-17,1996,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.624658,0.375342
6,Botox,Onabotulinumtoxina,3.305547e+08,AbbVie,BLA,1991-12-09,1991,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060274,0.939726
7,Brovana,NaN,NaN,NaN,NDA,2006-10-26,2006,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.180822,0.819178
8,Byetta,Exenatide,5.556382e+08,AstraZeneca,NDA,2005-04-28,2005,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.676712,0.323288
9,Byetta,Exenatide,5.556382e+08,AstraZeneca,NDA,2005-04-28,2005,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.676712,0.323288


In [31]:
pro_rate

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t41,t42,t43,t44,t45,t46,t47,t48,percent_remaining,percent_missing
0,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,Abraxane WW,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980822,0.019178
1,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,Abraxane US,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980822,0.019178
2,Abraxane,NaN,NaN,NaN,NDA,2005-01-07,2005,Abraxane Ex-US,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.980822,0.019178
3,Alimta,Pemetrexed Disodium,4.985018e+08,Lilly,NDA,2004-02-04,2004,Alimta WW,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.906849,0.093151
4,Alimta,Pemetrexed Disodium,4.985018e+08,Lilly,NDA,2004-02-04,2004,Alimta US,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.906849,0.093151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Xolair,Omalizumab,3.997580e+08,Roche,BLA,2003-06-20,2003,Xolair US,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.531507,0.468493
131,Xolair,Omalizumab,3.997580e+08,Roche,BLA,2003-06-20,2003,Xolair Ex-US,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.531507,0.468493
132,Xyrem,Sodium Oxybate,2.871436e+08,Jazz,NDA,2002-07-17,2002,Xyrem WW,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.457534,0.542466
133,Xyrem,Sodium Oxybate,2.871436e+08,Jazz,NDA,2002-07-17,2002,Xyrem US,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.457534,0.542466


In [29]:
def apply_discount(dataframe, discount_rate=0.1):
    '''
    '''
    for year in range(1,31):
        var = 't' + str(year)
        dataframe[var] = dataframe[var]/(1+discount_rate)**(year-0.5)
        
    return dataframe

final = apply_discount(pro_rate)

In [30]:
year_cols = [f't{i}' for i in range(1, 30+1)]
final[final['Proper Name'] == 'Adempas'][year_cols[10:]]

,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30
